In [ ]:
!pip install --upgrade pymilvus
!pip install git+https://github.com/zc277584121/graphrag.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
  Cloning https://github.com/zc277584121/graphrag.git to /tmp/pip-req-build-48mbd_bq
  Running command git clone --filter=blob:none --quiet https://github.com/zc277584121/graphrag.git /tmp/pip-req-build-48mbd_bq
  Resolved https://github.com/zc277584121/graphrag.git to commit c7263931eb9b3b6c7cd72fc11e4a6d4b11e6d48b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.6 MB/s eta 0

In [ ]:
file_path = 'sample_data/expanded_causal_dataset.json'  # Adjust the path to your expanded JSON


In [ ]:
!python -m graphrag.index --init --root ./graphrag_index


2024-10-10 10:15:40.993465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 10:15:41.029718: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 10:15:41.041550: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 10:15:42.778314: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing project at ./graphrag_index
⠋ GraphRAG Indexer 

In [ ]:
!pip install transformers
!pip install torch  # Necessary for running LLMs locally


In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

class CustomEmbedding:
    def __init__(self, model_name):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def get_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Use the [CLS] token's output as the embedding
        return outputs.last_hidden_state[:, 0, :].squeeze().numpy()


In [ ]:
pip install accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, infer_auto_device_map
from transformers import BitsAndBytesConfig

# Load the model in 8-bit precision
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Load model in 8-bit
)

# Offload parts of the model to the CPU to save RAM
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

# Initialize the model with offloading to CPU/disk to save RAM
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto",  # Automatically offloads to GPU/CPU
    quantization_config=quantization_config,  # Enable 8-bit quantization
    trust_remote_code=True
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RuntimeError: No GPU found. A GPU is needed for quantization.